In [17]:
import pandas as pd
import numpy as np

data_raw = pd.read_csv('../raw/지역사회건강조사 1차 전처리_2019.csv')

In [18]:
df = data_raw[['만나이','성별','행복감 지수','자치구']]

In [19]:
df_hi1=df[df['행복감 지수'] != 99]
df_hi2=df_hi1[df_hi1['행복감 지수'] != 77]

In [20]:
data_pbhlth_code = pd.read_excel('../raw/지역사회건강조사 2019년 원시자료 이용지침서 등/지역사회건강조사 2019 원시자료 참고사항_보건소정보, 세대유형.xlsx',header=3)
data_pbhlth_code = data_pbhlth_code[data_pbhlth_code['시도코드'] == 11][['보건소코드','보건소명']]
data_pbhlth_code["보건소명"] = data_pbhlth_code["보건소명"].str.replace('보건소', '', regex=True)
data_pbhlth_code.columns=['보건소코드','자치구']
list_region = list(data_pbhlth_code.to_dict()['자치구'].values()) + ['서울시']
list_region.reverse()

In [21]:
list_df = []
for value in list_region:
    temp = df_hi2['자치구'] == value
    list_df.append(df_hi2[temp])
list_df[0] = df_hi2

In [22]:
list_age = ['19~200','19~44','45~64','65~74','75~200']
list_sex = ['계','남','여']
idx_sex = []
idx_age = []
idx_region = []
value_hi = []
for i, value1 in enumerate(list_region):
    for j in list_age:
        for k in list_sex:            
            temp = list_df[i][list_df[i]['만나이']>=int(j.split('~')[0])]
            temp1 = temp[temp['만나이'] <= int(j.split('~')[1])]
            if k == '남':
                temp2 = temp1[temp1['성별'] == 1]
            elif k == '여':
                temp2 = temp1[temp1['성별'] == 2]
            else:
                temp2 = temp1
            temp3 = temp2['행복감 지수'].value_counts().to_dict()
            for m in range(1,11):
                if m not in temp3:
                    temp3[m] = 0
            temp_sum = 0
            for n in range(1,11):
                temp_sum += temp3[n]*n
            value_hi.append(round(temp_sum / len(temp2['행복감 지수']),2))
            if j == '19~200':
                idx_age.append('계')
            elif j == '75~200':
                idx_age.append('75 이상')
            else:
                idx_age.append(j)
            idx_sex.append(k)
            idx_region.append(value1)

In [23]:
df_final = pd.DataFrame({'행복감 지수':value_hi, '지역':idx_region,'연령별':idx_age,'성별':idx_sex})

df_final = df_final.set_index(['지역','연령별','성별'])

df_final

행복감 지수
지역  연령별   성별        
서울시 계     계     7.01
          남     7.05
          여     6.98
    19~44 계     7.12
          남     7.17
...              ...
강남구 65~74 남     7.10
          여     7.45
    75 이상 계     6.98
          남     7.00
          여     6.97

[390 rows x 1 columns]

In [26]:
df_final.to_csv("../raw/지역사회건강조사 (전처리된 데이터)_2019/지역사회건강조사 (행복감 지수)_2019.csv", mode='w',encoding = 'utf-8-sig')